In [63]:
# !pip install trectools
import pandas as pd
import trectools as trec

In [64]:
results_df = pd.read_csv("./MQ2007-agg/result_agg_functions.csv", sep="\t")
results_df.head(5)

,qid,docid,relevance,ranking_1,ranking_2,ranking_3,ranking_4,ranking_5,ranking_6,ranking_7,...,ranking_14_rank,ranking_15_rank,ranking_16_rank,ranking_17_rank,ranking_18_rank,ranking_19_rank,ranking_20_rank,rrf,squared_rrf,mrrf
0,10,GX000-00-0000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
1,10,GX000-24-12369390,1,107,107,85,0,113,110,110,...,8,3,8,0,7,6,14,0.230936,0.407611,1.429199
2,10,GX000-62-7863450,1,0,0,0,1,8,14,14,...,4,29,4,0,0,7,0,0.160114,0.228318,0.925124
3,10,GX016-48-5543459,1,0,0,111,4,1,3,3,...,10,26,10,3,1,18,7,0.225416,1.211949,1.295809
4,10,GX037-87-3082362,0,160,160,117,57,102,209,209,...,7,17,7,0,0,13,22,0.227372,0.366063,1.388782


In [65]:
qrel_df = results_df[["qid", "docid", "relevance"]].copy()
qrel_df['rank'] = qrel_df.groupby('qid')['relevance'].rank(method='dense', ascending=False).astype(int)
qrel_df['q0']  = 'Q0'
qrel_df['rel'] = qrel_df['relevance']
qrel_df['query'] = qrel_df['qid']
qrel_df = qrel_df[["qid","query", "q0", "docid", "rel"]]
print(qrel_df.head(5))
trec_qrel = trec.TrecQrel()
trec_qrel.qrels_data = qrel_df.copy()

   qid  query  q0              docid  rel
0   10     10  Q0   GX000-00-0000000    0
1   10     10  Q0  GX000-24-12369390    1
2   10     10  Q0   GX000-62-7863450    1
3   10     10  Q0   GX016-48-5543459    1
4   10     10  Q0   GX037-87-3082362    0


In [66]:
# Evaluate RRF
run_df_rrf = results_df[["qid", "docid", "rrf"]].copy()
run_df_rrf['rank'] = run_df_rrf.groupby('qid')['rrf'].rank(method='dense', ascending=False).astype(int)
run_df_rrf['q0']  = 'Q0'
run_df_rrf['score'] = run_df_rrf['rrf']
run_df_rrf['system'] = 'rrf'
run_df_rrf['query'] = run_df_rrf['qid']
run_df_rrf = run_df_rrf[["qid","query","q0", "docid", "score", "system"]]
# print(run_df_rrf.head(5))

# RRF Evaluation
run_rrf = trec.TrecRun()
run_rrf.run_data = run_df_rrf.copy()
rrf_trec_eval = trec.TrecEval(run_rrf, trec_qrel)
print(f"rrd:{rrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    print(f"precision@{i}:{rrf_trec_eval.get_precision(depth=i)}")
    print(f"map@{i}:{rrf_trec_eval.get_map(depth=i)}")
    


precision@1:0.30141843971631205
map@1:0.024912103443446538


TypeError: get_relevant_retrieved_documents() got an unexpected keyword argument 'depth'

In [67]:
# Evaluate SRRF
run_df_srrf = results_df[["qid", "docid", "squared_rrf"]].copy()
run_df_srrf['rank'] = run_df_srrf.groupby('qid')['squared_rrf'].rank(method='dense', ascending=False).astype(int)
run_df_srrf['q0']  = 'Q0'
run_df_srrf['score'] = run_df_srrf['squared_rrf']
run_df_srrf['system'] = 'srrf'
run_df_srrf['query'] = run_df_srrf['qid']
run_df_srrf = run_df_srrf[["qid","query","q0", "docid", "score", "system"]]
# print(run_df_srrf.head(5))

# SRRF Evaluation
run_srrf = trec.TrecRun()
run_srrf.run_data = run_df_srrf.copy()
srrf_trec_eval = trec.TrecEval(run_srrf, trec_qrel)
print(f"rrd:{srrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    # print(f"ndcg@{i}:{srrf_trec_eval.get_ndcg(depth=i)}")
    print(f"precision@{i}:{srrf_trec_eval.get_precision(depth=i)}")
    print(f"map@{i}:{srrf_trec_eval.get_map(depth=i)}")
    

precision@1:0.19444444444444445
map@1:0.014664116447064002


TypeError: get_relevant_retrieved_documents() got an unexpected keyword argument 'depth'

In [68]:
# Evaluate SRRF
run_df_mrrf = results_df[["qid", "docid", "mrrf"]].copy()
run_df_mrrf['rank'] = run_df_mrrf.groupby('qid')['mrrf'].rank(method='dense', ascending=False).astype(int)
run_df_mrrf['q0']  = 'Q0'
run_df_mrrf['score'] = run_df_mrrf['mrrf']
run_df_mrrf['system'] = 'mrrf'
run_df_mrrf['query'] = run_df_mrrf['qid']
run_df_mrrf = run_df_mrrf[["qid","query","q0", "docid", "score", "system"]]
print(run_df_mrrf.head(5))

# SRRF Evaluation
run_mrrf = trec.TrecRun()
run_mrrf.run_data = run_df_mrrf.copy()
mrrf_trec_eval = trec.TrecEval(run_mrrf, trec_qrel)
print(f"rrd:{mrrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    print(f"precision@{i}:{mrrf_trec_eval.get_precision(depth=i)}")
    print(f"map@{i}:{mrrf_trec_eval.get_map(depth=i)}")
    

   qid  query  q0              docid     score system
0   10     10  Q0   GX000-00-0000000  0.000000   mrrf
1   10     10  Q0  GX000-24-12369390  1.429199   mrrf
2   10     10  Q0   GX000-62-7863450  0.925124   mrrf
3   10     10  Q0   GX016-48-5543459  1.295809   mrrf
4   10     10  Q0   GX037-87-3082362  1.388782   mrrf
precision@1:0.26891252955082745
map@1:0.02134808547993294


TypeError: get_relevant_retrieved_documents() got an unexpected keyword argument 'depth'